In [14]:
%pip install --quiet pydantic chromadb langchain langchain_experimental langchain_openai langchain_community


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
from pathlib import Path
import json

product_path = Path("../initial-training-sets/datasets/dataset_products.json")
product_images_path = Path("../initial-training-sets/datasets/images/")
product_data = json.loads(product_path.read_text())

from datetime import date
from uuid import UUID
from pydantic import BaseModel
from typing import List

class RawReview(BaseModel):
    id: UUID
    review_ref: str
    product_id: UUID
    review_content: str
    review_title: str
    date_written: date
    product_asin: str
    helpful_count: int
    rating_given: int
    review_page_url: str

class RAWProduct(BaseModel):
    id: UUID
    name: str
    description: str
    product_asin: str
    overall_ratings: float
    total_customers_that_rated: int
    price: float
    currency: str
    category: str
    sub_category: str
    product_page_url: str
    image_url: str
    reviews: List[RawReview]
    

from langchain_core.documents import Document

class ProductCombinedInformation:
    
    product: RAWProduct
    
    def __init__(self, product: RAWProduct) -> None:
        self.product = product
        
    def get_document(self):
        return Document(page_content=self.info(), id=str(self.product.id), metadata={
            "product_id": self.product.id,
            "product_asin": self.product.product_asin,
            "image_url": self.product.image_url,
            "name": self.product.name
        })
    
    
    def info(self):
        return (
            f"Product ID: {str(self.product.id).strip()} \n"
            f"Product Name: {str(self.product.name).strip()} \n"
            f"Product Description: {str(self.product.description).strip()} \n"
            f"Product Asin: {self.product.product_asin} \n"
            f"Overall Ratings {self.product.overall_ratings} \n"
            f"Total Customers that rated: {self.product.total_customers_that_rated} \n"
            f"Pric: {self.product.currency}{self.product.price} \n"   
        )
        
    def image_path(self):
        return f"{product_images_path}/{self.product.product_asin}.png"
    

In [17]:

product_summary_model_name = "llama3.1"

# product_summary_model_name = "gpt-4o"
# image_description_model_name = "gpt-4o"
# text_embedding_model_name = ""

### PID: Product Image Description

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOllama
from PIL import Image
from IPython.display import HTML, display
import base64

def encode_image(image_path: str):
    """Returns the base64 string for the image"""
    
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def describe_image(img_base64, prompt, model_name: str, image_format: str = "png"):
        
    
    """
    Describes the image provided. The prompt is used to guide the model on what
    to do with the description and what it should do.

    Raises:
        ValueError: If the wrong model name is provided.
        Exception: If any error occur

    Returns:
        str: The description generated for the image
    """
    
    if model_name not in [ "llava", "gpt-4o" ]:
        raise ValueError("Please provide either llava or gpt-4o")
    
    image_url = f"data:image/{image_format};base64,{img_base64}"
    
    if model_name == "gpt-4o":
        chat = ChatOpenAI(model=model_name, max_tokens=1024, temperature=0)
        msg = chat.invoke([
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url
                        }
                    }
                ]
            )
        ])
    elif model_name == "llava":
        chat = ChatOllama(model=model_name, num_ctx=1024)
        msg = chat.invoke([
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url
                        }
                    }
                ]
            )
        ])
    else:
        raise Exception("Unknown model name.")
        
    return msg.content


def plt_img_base64(img_base64):
  """Display base64 encoded string as image"""

  # Create an HTML img tag with the base64 string as the source
  image_html = f'<img width="200px" height="200px" src="data:image/jpeg;base64,{img_base64}" />'
  display(HTML(image_html))

### PCI: Product Combined Info

In [139]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough

def summarise_product_info(product_info: str, model_name: str):
    
    if model_name not in [ "llava", "gpt-4o", "llama3", "llama3.1", "phi3", "mistral" ]:
        raise ValueError("Please provide either llava or gpt-4o")
    
    prompt_template = ("You are an assistant tasked with summarizing tables and "
                    "text for retrieval. These summaries will be embedded and used " 
                    "to retrieve the raw text. Give a concise summary of the product. "
                    "Ensure you include important details "
                    "that is well optimized for retrieval.\n"
                    f"Product: {product_info}\n"
                    "Your response should start with the product name."
                    )
    
    prompt = ChatPromptTemplate.from_template(prompt_template)
    
    if model_name == "gpt-4o":
        model = ChatOpenAI(model=model_name, max_tokens=1024, temperature=0)
    else:
        model = ChatOllama(model=model_name, temperature=0)
    
        
    # { "product_info": RunnablePassthrough() } is same as lambda x: x
    # basically do not modify what the user entered, pass into the key product_info
    summarise_chain = { "product_info": RunnablePassthrough() } | prompt | model | StrOutputParser()
    summary = summarise_chain.invoke(product_info)
    return summary
    

In [20]:
max_products_per_category = 20

product_data.keys()

dict_keys(['MEN_FASHION_WITH_REVIEWS', 'SKIN_CARE_WITH_REVIEWS', 'WOMEN_FASHION_NO_REVIEWS'])

### Men Fashion

In [21]:
products = product_data['MEN_FASHION_WITH_REVIEWS']['products'][0:max_products_per_category]


# Remove Duplicates
filtered_products = []
checked_products = []

print("Before Removing Duplicates ", len(products))
for product in products:
    if product['product_asin'] not in checked_products:
        print(product['product_asin'])
        filtered_products.append(product)
        checked_products.append(product['product_asin'])

products = filtered_products

print("After Removing Duplicates ", len(products))

combined_product_infos = [ ProductCombinedInformation(product=RAWProduct.model_validate(product)) for product in products ]

# Summarise Product Info = PCIS
product_info_summaries = [summarise_product_info(product_info.info(), model_name=product_summary_model_name) for product_info in combined_product_infos]


# for pis in product_info_summaries:
#     print(pis, "\n\n")

Before Removing Duplicates  20
B0C812K6RR
B07BN6KCZT
B001BEAWXY
B07BSZXWWT
B01DBU6MQ6
B0BHMB8F2Q
B0BRNFVSCF
B098PCGH4F
B004LXWRH6
B09QRPLWJ3
B01B1D12B0
B09C5FHR99
B091SJ27ND
B09L7KYXL5
B07WZDCNSZ
B09HXLDG5B
After Removing Duplicates  16


In [184]:

# Describe the Product Image = PID
image_prompt = f"""
Describe and summarise the characteristics of the product you are looking at. Start your response with: `The image is a product ... `. Return back plain text no markdown.
"""

image_description_model_name = "gpt-4o"
image_prompt = f"""
Describe and summarise the characteristics of the product you are looking at. Start your response with: `The image is a product ... `. 
Give a short summary of what the product can be used for.
Return back plain text no markdown.
"""
product_image_descriptions = [describe_image(encode_image(image_path=product_info.image_path()), prompt=image_prompt, model_name=image_description_model_name) for product_info in combined_product_infos ]

for index, pid in enumerate(product_image_descriptions):
    print(index, pid, "\n\n")

0 The image is a product showcasing a pair of athletic shoes. These shoes feature a modern design with a breathable mesh upper, providing ventilation and comfort. The sole is thick and cushioned, likely offering good shock absorption and support for various activities. The shoes come in multiple color options, including a gradient of purple to pink, blue to black, and solid black. They have a lace-up closure for a secure fit and a pull tab at the heel for easy wearing.

These athletic shoes can be used for a variety of physical activities such as running, walking, gym workouts, and casual wear. Their design suggests they are suitable for both sports and everyday use, providing comfort and style. 


1 The image is a product showcasing a men's long-sleeve button-up shirt. The shirt is gray in color and features a classic collar, buttoned front, and two chest pockets with buttoned flaps. The material appears to be soft and comfortable, suitable for casual or semi-casual wear. This shirt c

In [185]:
product_info_summaries[2], product_image_descriptions[2]

('Here is a concise summary of the product, optimized for retrieval:\n\n**Men Boxer Short Trunks Stretch Cotton Pack of 3**\n\n* **Product Details**: Calvin Klein boxer shorts with CK branding, designed for everyday wear and sports.\n* **Material Composition**: 95% Cotton, 5% Elastane (Spandex).\n* **Care Instructions**: Machine Washable.\n* **Key Features**:\n\t+ Comfortable fit due to stretchy fabric.\n\t+ Classic elastic waistband with logo.\n\t+ Soft and breathable cotton stretch material.\n* **Brand Information**: Calvin Klein lifestyle brand, known for minimalist aesthetics and timeless designs.\n* **Product Specifications**:\n\t+ Package Dimensions: 19.56 x 13.21 x 3.3 cm; 100 Grams.\n\t+ Date First Available: June 18, 2023.\n\t+ Manufacturer: Calvin Klein.\n\t+ Item Model Number: 0000U2662G.\n* **Customer Reviews**: 4.6 out of 5 stars (36,088 ratings).\n* **Price**: £28.0.\n\nThis summary includes the most important details about the product, making it easy to retrieve and comp

### Create the Multi Vector Retriever

In [186]:
# Vector Store for Document Embeddings
from langchain_chroma import Chroma
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_openai import OpenAIEmbeddings

from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

from chromadb.errors import InvalidDimensionException

def get_base_retriever(reset_collection: bool = False):
        
    # OpenCLIP: https://github.com/mlfoundations/open_clip
    # Larger and more performant
    # model_name = "ViT-g-14"
    # checkpoint = "laion2b_s34b_b88k"

    # # Smaller less performant
    # model_name = "ViT-B-32"
    # checkpoint = "laion2b_s34b_b79k"

    # # Default
    model_name = "ViT-H-14"
    checkpoint = "laion2b_s32b_b79k"

    text_embedding_model_name = "nomic-embed-text"

    # for nomic-embed, embed_instruction is `search_document` to embed documents for RAG and `search_query` to embed the question
    underlying_embedding = OllamaEmbeddings(model=text_embedding_model_name, embed_instruction="search_document", query_instruction="search_query")
    # underlying_embedding = OpenAIEmbeddings()
    reset_collection = True

    store = LocalFileStore("./embeddings-cache")

    cached_embedder = CacheBackedEmbeddings.from_bytes_store(
        underlying_embeddings=underlying_embedding, document_embedding_cache=store, namespace=underlying_embedding.model
    )


    collection_name = f"fashion_store_mrag_v_{underlying_embedding.model}"
    try:
        products_vectorstore = Chroma(
            collection_name=collection_name ,
            embedding_function=cached_embedder,
            # https://docs.trychroma.com/guides#changing-the-distance-function
            # Cosine, 1 means most similar, 0 means orthogonal, -1 means opposite
            collection_metadata={"hnsw:space": "cosine"}, # l2 is the default
            # embedding_function=OpenCLIPEmbeddings(model=None, preprocess=None, tokenizer=None, model_name=model_name, checkpoint=checkpoint)
        )
    except InvalidDimensionException:
        Chroma().delete_collection()
        products_vectorstore = Chroma(
            collection_name=collection_name ,
            embedding_function=cached_embedder,
            collection_metadata={"hnsw:space": "cosine"}, # l2 is the default
        )
        
    if reset_collection:
        products_vectorstore.reset_collection()
        
    return products_vectorstore


In [187]:
import json
import os
from pathlib import Path

images_path = "../initial-training-sets/datasets/images"

from langchain.retrievers.multi_vector import MultiVectorRetriever, SearchType
from langchain.storage import InMemoryStore
from langchain_core.documents import Document

from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=4000)

n_docs_to_retrieve = 15
relevant_threshold_score = 0.5

def create_multi_vector_retriever(
        embeddings_vectorsotre, 
        product_combined_info_summaries: List[str],  # PCIS -> Product Combined Info Summary
        product_image_descriptions: List[str],  # PID
        product_combined_infos: List[ProductCombinedInformation] # PCI objects
    ):
    
    # Initialise the document storage layer
    document_store = InMemoryStore()
    id_key = "product_id"
    
    retriever = MultiVectorRetriever(
        vectorstore=embeddings_vectorsotre,
        docstore=document_store,
        id_key=id_key,
        search_type=SearchType.similarity_score_threshold,
        search_kwargs={
            # below works only for similarity_score_threshold search_type
            "score_threshold": relevant_threshold_score, # close to 0 means most dissimilar, 1 means most similar
            "k": n_docs_to_retrieve
        }
    )
    
    # Helper function to add documents into the vector and the doument store
    def add_documents_with_splits(retriever: MultiVectorRetriever, summary_to_embeds: List[str], combined_product_infos: List[ProductCombinedInformation]):
        product_ids=[ str(product_info_obj.product.id) for product_info_obj in combined_product_infos ]
        data = list(zip(product_ids, summary_to_embeds, combined_product_infos))
               
        docs = []
        parent_docs_contents = []
        
        for single_item in data:
            product_id, content_to_embed, product_info = single_item
            for content_chunk in splitter.split_text(content_to_embed):
                docs.append(Document(
                    page_content=content_chunk,
                    metadata={
                        id_key: product_id,
                        "id": product_id,
                        "name": product_info.product.name,
                        "product_asin": product_info.product.product_asin
                    }
                ))
                parent_docs_contents.append(
                    Document(
                        page_content=product_info.info(),
                        metadata={
                            id_key: product_info.product.id,
                            "id": product_info.product.id,
                            "name": product_info.product.name,
                            "product_asin": product_info.product.product_asin
                        }
                    ) 
                )
        
        assert len(docs) == len(parent_docs_contents)
        
        retriever.vectorstore.add_documents(docs)
        retriever.docstore.mset(list(zip(product_ids, parent_docs_contents)))
        
        return retriever
    
    # Helper function to add documents into the vector and the doument store
    def add_documents_no_split(retriever: MultiVectorRetriever, summary_to_embeds: List[str], combined_product_infos: List[ProductCombinedInformation]):
        product_ids=[ str(product_info_obj.product.id) for product_info_obj in combined_product_infos ]
        data = list(zip(product_ids, summary_to_embeds, combined_product_infos))
               
        docs = []
        parent_docs_contents = [
                    Document(
                        page_content=product_info.info(),
                        metadata={
                            id_key: product_info.product.id,
                            "id": product_info.product.id,
                            "name": product_info.product.name,
                            "product_asin": product_info.product.product_asin
                        }
                    ) 
                    for product_info in combined_product_infos
        ]
        
        for single_item in data:
            product_id, content_to_embed, product_info = single_item
            docs.append(Document(
                page_content=content_to_embed,
                metadata={
                    id_key: product_id,
                    "id": product_id,
                    "name": product_info.product.name,
                    "product_asin": product_info.product.product_asin
                }
            ))
        
        assert len(docs) == len(parent_docs_contents)
        
        retriever.vectorstore.add_documents(docs, ids=product_ids )
        retriever.docstore.mset(list(zip(product_ids, parent_docs_contents)))
        
        return retriever
    
    if product_combined_info_summaries:
        add_documents_no_split(
            retriever=retriever,
            summary_to_embeds=product_combined_info_summaries,
            combined_product_infos=product_combined_infos
        )
        
    if product_image_descriptions:
        add_documents_no_split(
            retriever=retriever,
            summary_to_embeds=product_image_descriptions,
            combined_product_infos=product_combined_infos
        )
        
    return retriever
        
        

In [188]:
products_vectorstore = get_base_retriever(reset_collection=True)
retriever = create_multi_vector_retriever(
    embeddings_vectorsotre=products_vectorstore, 
    product_combined_info_summaries=product_info_summaries, 
    product_image_descriptions=product_image_descriptions,
    product_combined_infos=combined_product_infos, 
)

# retriever_query = "I dark coloured casual pants" # description only in the image
# retriever_query = "high tech glasses" # description only in the image
# retriever_query = "queen" # description only in the image
# retriever_query = "pants with zipper fly" # description only in the image

# docs = retriever.invoke(retriever_query)

# for doc in docs:
#     print(doc.page_content)
    

# print("----")
# # retriever.vectorstore.similarity_search_with_relevance_scores(retriever_query, k=3)
# docs = retriever.vectorstore.similarity_search_with_relevance_scores(retriever_query, k=n_docs_to_retrieve)
# for doc, score in docs:
#     print(score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin') )


### Synthesize Answer / Pass docs to LLM

In [189]:
load_dotenv()


True

In [190]:
from langchain_community.chat_models.ollama import ChatOllama

from langchain.output_parsers.pydantic import  PydanticOutputParser
from pydantic import BaseModel, Field

class ProductDetail(BaseModel):
  product_id: UUID = Field(description="ID of the product recommended to the user")
  detail: str = Field(description="Assistant message to the user about the product")
  title: str = Field(description="Price of the product")

class AssistantResponse(BaseModel):
  response: str = Field(description="The start of the assistant's answer.")
  products: List[ProductDetail] = Field(description="A collection of products being recommended for the user")


parser = PydanticOutputParser(pydantic_object=AssistantResponse)

answer_generation_model = "gpt-4o"
# answer_generation_model = "llama3.1"
# answer_generation_model = "mistral"

if "gpt" not in answer_generation_model:
  model = ChatOllama(model=answer_generation_model, temperature=0, num_ctx=1024)
else:
  model = ChatOpenAI(model=answer_generation_model, temperature=0)


from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever

# Retrieval Chain: summarise history + current query/input to fetch relevant documents
history_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        SystemMessage(content="generate a search query to look up in order to get information relevant to the conversation."),
        ("user", "{input}"),
    ]
)
history_chain = create_history_aware_retriever(
  llm=model, retriever=retriever, prompt=history_prompt
)

# Answer Synthesising
system_message = """
                  \n{format_instructions}\n
                  
                  You work as a stylish and Gen-Z stylish who is working as a customer assistance in a fashion store.
                  You help customers to answer general questions about fashion and then tries to recommend a product 
                  if there is any that matches in the inventory below. If no inventory, do not recommend any product 
                  but you can provide general advice on fashion for them.
                  
                  <INVENTORY_START>
                  {context}
                  <INVENTORY_ENDS>
                  
                  Do not provide any link. Always reply as if you are talking to customer directly.
                  
                  In situation where there are multiple products in the inventory, explain how each of them can be helpful.
                  
                  There are times the inventory will be empty, assure the user we will get the product that would match their query if it is something a 
                  fashion store can provide as a service.
                  
                  All your response will be parsed as a JSON not markdown, which will fail if it is not in JSON format.
                """
                
                #   You should format your response in this format. 
                #   {{
                #     'response_summary': "",
                #     "products": [
                #       {{ "product_asin": "", "detail": "" }}
                #     ]
                #   }}
                  
                # """
format_instructions = parser.get_format_instructions()
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message ),
    MessagesPlaceholder(variable_name="chat_history"), # empty for now
    ("human", "{input}" ),
])

# a chain that allows us pass a list of docs to a model
stuff_documents_and_prompts_chain = create_stuff_documents_chain(llm=model, prompt=prompt)



# a chain to use the retriever we have to retireve the relevant docs and then pass them to the chain that has the llm+prompt
chain = create_retrieval_chain(
  retriever=history_chain, # We can either pass a retriever or a chain
  combine_docs_chain=stuff_documents_and_prompts_chain
)

# user_question = "The summer is here, what item can I buy to add to my accessory? Glasses, hats etc"
user_question = "The summer is here, what item can I buy to add to my accessory?"

# The documents fetched does not meet our threshold for relevance scores, so it wil be empty. Inventory is empty!
docs = retriever.vectorstore.similarity_search_with_relevance_scores(user_question, k=n_docs_to_retrieve)
for doc, score in docs:
    print(score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin'))


answer = chain.invoke({"input": user_question, "chat_history": [], 
                       "format_instructions": format_instructions}
                      )
print(answer['answer'])
# import json
# json.loads(answer['answer'])

0.5956048369407654 True  -  B01B1D12B0
0.5841009616851807 True  -  B09HXLDG5B
0.5731948018074036 True  -  B09QRPLWJ3
0.5659385919570923 True  -  B09C5FHR99
0.5603852868080139 True  -  B09L7KYXL5
0.5574619770050049 True  -  B0C812K6RR
0.55238276720047 True  -  B07BSZXWWT
0.5501329898834229 True  -  B004LXWRH6
0.5491613149642944 True  -  B0BRNFVSCF
0.546113133430481 True  -  B01DBU6MQ6
0.5444117188453674 True  -  B098PCGH4F
0.5350827574729919 True  -  B001BEAWXY
0.5350075364112854 True  -  B07WZDCNSZ
0.530100405216217 True  -  B0BHMB8F2Q
0.5212832689285278 True  -  B07BN6KCZT
{
  "response": "Summer is the perfect time to add some stylish accessories to your wardrobe! I have a great recommendation for you.",
  "products": [
    {
      "product_id": "96f87d3b-5202-4ada-8b60-a1326719302f",
      "detail": "These Men's Retro Driving Polarized Sunglasses are perfect for the summer. They offer 100% UV400 protection, reducing the risk of visual fatigue and skin aging due to UV exposure. The u

#### What if we have multiple products?

In [191]:
user_question = "I need to buy trousers! I need like three."
# user_question = "I want to buy some new trousers, I need three"

# The documents fetched does not meet our threshold for relevance scores, so it wil be empty. Inventory is empty!
docs = retriever.vectorstore.similarity_search_with_relevance_scores(user_question, k=n_docs_to_retrieve)
for doc, score in docs:
    print(score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin'))


chain.invoke({"input": user_question, "chat_history": [], "format_instructions": format_instructions})

0.5687891840934753 True  -  B09HXLDG5B
0.5633045434951782 True  -  B091SJ27ND
0.5566449761390686 True  -  B001BEAWXY
0.5395516157150269 True  -  B07BSZXWWT
0.5300022959709167 True  -  B09C5FHR99
0.5279671549797058 True  -  B0BHMB8F2Q
0.5128794312477112 True  -  B0BRNFVSCF
0.495597243309021 False  -  B07WZDCNSZ
0.48970937728881836 False  -  B09QRPLWJ3
0.4829484224319458 False  -  B07BN6KCZT
0.47927016019821167 False  -  B09L7KYXL5
0.4703163504600525 False  -  B0C812K6RR
0.4471980333328247 False  -  B01DBU6MQ6
0.44163596630096436 False  -  B098PCGH4F
0.43703150749206543 False  -  B01B1D12B0


{'input': 'I need to buy trousers! I need like three.',
 'chat_history': [],
 'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"ProductDetail": {"properties": {"product_id": {"description": "ID of the product recommended to the user", "format": "uuid", "title": "Product Id", "type": "string"}, "detail": {"description": "Assistant message to the user about the product", "title": "Detail", "type": "string"}, "title": {"description": "Price of the product", "title": "Title", "type": "string"}}, "required": ["product_id", "detail", "title"], "title": "Pr

#### Thoughts on improving our Retriever with MultiQuery?

In a situation with the question below, we will need to introduce a multi-query to ask the questions in different ways we can have more products to 

Question: "The summer is here, what item can I buy to add to my accessory?"

Adding hats and glasses will improve the answer as shown below

In [192]:
question = "The summer is here, what item can I buy to add to my accessory?"
chain.invoke({"input": question, "chat_history": [], "format_instructions": format_instructions})

{'input': 'The summer is here, what item can I buy to add to my accessory?',
 'chat_history': [],
 'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"ProductDetail": {"properties": {"product_id": {"description": "ID of the product recommended to the user", "format": "uuid", "title": "Product Id", "type": "string"}, "detail": {"description": "Assistant message to the user about the product", "title": "Detail", "type": "string"}, "title": {"description": "Price of the product", "title": "Title", "type": "string"}}, "required": ["product_id", "detail", "

#### Image + Text Customer Prompt

Implement how we can query the RAG system with a text and a picture

In [193]:
product_asin = "B07BN6KCZT" # grey shirt
product_asin = "B01B1D12B0" # sun glasses
query_image_path = f"../initial-training-sets/MEN_FASHION_WITH_REVIEWS/images/{product_asin}.png"

# Encode the image
query_image_encoded = encode_image(query_image_path)
# Summarise the image
describe_query_image_prompt = """Describe what you are seeing to enable us to retrieve an items that resembles this product"""
# Compose User Prompt and Pass the image+prompt to the conversational chain
query_image_description = describe_image(query_image_encoded, prompt=describe_query_image_prompt, model_name=image_description_model_name)

print("Image Described", query_image_description)

# Test this with out vector store
image_related_docs = retriever.vectorstore.similarity_search_with_relevance_scores(query_image_description, k=n_docs_to_retrieve)
for doc, score in image_related_docs:
    print(score, doc.page_content)
    

customer_prompt = "I am looking to buy a set of glasses that looks like this and a trouser that goes well with Suits."
customer_prompt = "I am looking to buy a set of glasses that looks like this and a trouser that goes well with it"

customer_prompt_formatted = f"""
Customer Message: {customer_prompt}.
<DESCRIPTION_START>{query_image_description}<DESCRIPTION_ENDS>
"""

chain.invoke({"input": customer_prompt_formatted, "chat_history": [], "format_instructions": format_instructions})

Image Described The product in the image is a pair of sunglasses with a sleek, modern design. Here are the key features:

1. **Frame**: The frame is black and has a glossy finish. It appears to be made of metal or a high-quality plastic material.
2. **Lens Shape**: The lenses are rectangular with slightly rounded edges, giving them a classic yet contemporary look.
3. **Lens Color**: The lenses are reflective, showing a black and white cityscape. This reflective quality suggests that the lenses might be polarized or mirrored.
4. **Temple Design**: The temples (arms) of the sunglasses are also black and have a textured pattern on the inner side, which could be for added grip or aesthetic purposes.
5. **Nose Pads**: The sunglasses have adjustable nose pads, which are typically found on higher-end or more comfortable sunglasses.
6. **Branding**: There is some text on the temples, which might be the brand name or model number.

To find a similar product, look for black, rectangular sunglass

{'input': '\nCustomer Message: I am looking to buy a set of glasses that looks like this and a trouser that goes well with it.\n<DESCRIPTION_START>The product in the image is a pair of sunglasses with a sleek, modern design. Here are the key features:\n\n1. **Frame**: The frame is black and has a glossy finish. It appears to be made of metal or a high-quality plastic material.\n2. **Lens Shape**: The lenses are rectangular with slightly rounded edges, giving them a classic yet contemporary look.\n3. **Lens Color**: The lenses are reflective, showing a black and white cityscape. This reflective quality suggests that the lenses might be polarized or mirrored.\n4. **Temple Design**: The temples (arms) of the sunglasses are also black and have a textured pattern on the inner side, which could be for added grip or aesthetic purposes.\n5. **Nose Pads**: The sunglasses have adjustable nose pads, which are typically found on higher-end or more comfortable sunglasses.\n6. **Branding**: There is

#### Improving Query with Metadata search via Self-Query
Allowing the model to see the reviews for the products

https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/self_query/
https://python.langchain.com/v0.1/docs/integrations/retrievers/self_query/chroma_self_query/